## Sarcasm Detection

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf 
from tensorflow import keras
import json

In [17]:
json_sar=[]
with open('Sarcasm_Headlines_Dataset.json','r',encoding='utf-8') as sarcasm:
    for lines in sarcasm:
        json_sar.append(json.loads(lines))

In [20]:
train_data,train_label=[],[]
for items in json_sar:
    train_data.append(items['headline'])
    train_label.append(items['is_sarcastic'])


In [21]:
train_data=np.asarray(train_data)
train_label=np.asarray(train_label)

In [22]:
train_data.shape

(26709,)

In [24]:
train_label.shape

(26709,)

In [25]:
from sklearn.model_selection import train_test_split

In [29]:
train_cat,test_cat,train_sarc,test_sarc=train_test_split(train_data,train_label,test_size=0.2)

In [31]:
train_cat.shape,test_cat.shape

((21367,), (5342,))

In [32]:
train_cat[2]

'goth kid builds scary-ass birdhouse'

In [33]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [34]:
token=Tokenizer()
token.fit_on_texts(train_cat)
word_index=token.word_index
vocab_size=len(word_index)

In [35]:
vocab_size

26611

In [36]:
train_seq=token.texts_to_sequences(train_cat)
test_seq=token.texts_to_sequences(test_cat)

In [37]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [38]:
train_pad=pad_sequences(train_seq,maxlen=15,padding='post')
test_pad=pad_sequences(test_seq,maxlen=15,padding='post')

In [49]:
train_pad.shape,test_pad.shape,test_sarc.shape

((21367, 15), (5342, 15), (5342,))

In [50]:
test_sarc[:10]

array([0, 1, 1, 0, 0, 1, 0, 1, 1, 1])

In [40]:
from tensorflow.keras.layers import Embedding,GlobalAveragePooling1D,Dense

In [41]:
model=keras.Sequential()
model.add(Embedding(input_dim=vocab_size+1,output_dim=64,input_length=15))
model.add(GlobalAveragePooling1D())
model.add(Dense(units=128,activation='relu'))
model.add(Dense(units=1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 15, 64)            1703168   
_________________________________________________________________
global_average_pooling1d (Gl (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 128)               8320      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 1,711,617
Trainable params: 1,711,617
Non-trainable params: 0
_________________________________________________________________


In [51]:
model.fit(train_pad,train_sarc,validation_data=(test_pad,test_sarc),epochs=5)

Train on 21367 samples, validate on 5342 samples
Epoch 1/5
21367/21367 [==============================] - 17s 818us/sample - loss: 0.4173 - accuracy: 0.7954 - val_loss: 0.3352 - val_accuracy: 0.8545
Epoch 2/5
21367/21367 [==============================] - 16s 729us/sample - loss: 0.1964 - accuracy: 0.9227 - val_loss: 0.3534 - val_accuracy: 0.8531
Epoch 3/5
21367/21367 [==============================] - 15s 693us/sample - loss: 0.1026 - accuracy: 0.9646 - val_loss: 0.4346 - val_accuracy: 0.8487
Epoch 4/5
21367/21367 [==============================] - 15s 689us/sample - loss: 0.0576 - accuracy: 0.9812 - val_loss: 0.5352 - val_accuracy: 0.8471
Epoch 5/5
21367/21367 [==============================] - 15s 696us/sample - loss: 0.0340 - accuracy: 0.9888 - val_loss: 0.6623 - val_accuracy: 0.8356
